In [1]:
import torch

# 数据加载

数据载入类：torch.utils.data.DataLoader
参数：
 - dataset: torch.utils.data.Dataset 类的实例
 - batch_size: 迷你批次的大小
 - shuffle： 数据会不会被随机打乱
 - samlpler：自定义的数据采样器，shuffle为true会生成一个默认的。自定义需要设置shuffle为false，每次迭代返回一个数据的下标索引
 - batch_sampler： 返回一个迷你批次的数据索引
 - num_workers: 数据载入器使用的进程数目
 - collate_fn: 定义如何把一批dataset 的实例转换为包含迷你批次数据的张量
 - pin_memory： 把数据转移到和GPU内存相关联的CPU内存
 - drop_last: 是否要把最后一个迷你批次的数据丢掉，因为它通常小于设置的batch_size
 - timeout: 如果大于0，决定在多进程情况下对数据的等待时间
 - worker_init_fn: 每个数据载入的子进程开始时运行的函数

数据集类型主要有两种：
- 映射类型的数据集 torh.utils.data.Dataset
- 可迭代类型的数据集 不需要实现__getitem__和__len__

In [2]:
class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset, self).__init__()
        assert end > start, "this example code only works with end >= start"
        self.start = start
        self.end = end
    
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None: # 单进程数据载入
            iter_start = self.start
            iter_end = self.end
        else:
            per_workder = int(math.ceil((self.end-self.start) / float(worker_info.num_works)))
            worker_id = worker_info.worker_id
            iter_start = self.start + worker_id * per_workder
            iter_end = min(iter_start + per_workder, self.end)
        
        return iter(range(iter_start, iter_end))

## 模型和张量的序列化和反序列化

In [ ]:
torch.save(obj, f, pickle_module=pickle, pickle_protocol=2)
torch.load(f, map_location=None, pickle_module=pickle, **pickle_load_args)

默认的序列化库是pickle,pickle_protocol是序列化协议

## 分布式数据并行化

torch.distributed 包含用于分布式计算API，主要支持：
- Gloo 支持CPU，GPU，对GPU支持差
- MPI 只支持CPU
- NCCL 只支持GPU

为了启动不同的进程，首先要对所有计算进程进行初始化。

In [ ]:
torch.distributed.init_process_group(backend, init_method=None, timeout=datetime.timedelta(0, 1800), world_size=-1, rank=-1, store=None, group_name='')

- backend 分别传入三个字符串 gloo， mpi， nccl
- init_method 值为None，默认采用 env:// 来启动进程，即每个进程会从当前的环境变量中获取当前节点和其他节点的信息。 其他方法可以通过`tcp://ip:port`的方式来哦获取其他进程的信息
- timeout 操作的超时时间，只对gloo有效
- store 设置了，init_method会失效，通过传入字典的键值对来指定节点信息
- word_size 进程数目
- rank 当前进程的序号
- group_name 进程组的名字已废弃

In [ ]:
# 分布式数据加载
train_dataset = datasets.ImageFolder(traindir, transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]
))

train_sampler =  torch.utils.data.distributed.DistributedSampler(train_dataset)
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=args.batch_size,
    shuffle=(train_simpleer is None),
    num_workers=args.workers,
    pin_memory=True,
    sampler=train_sampler
)

In [ ]:
torch.nn.parallel.DistributedDataParallel(module=module, device_ids=None, output_device=None, dim=0, broadcast_buffers=True,
process_group=None, bucket_cap_mb=25, find_unused_parameters=False, check_reduction=False)

- module 需要被分布式数据并行化的模型
- device_ids 分布式GPU的id
- output_device 输出的设备
- dim 分割的维度
- broadcast_buffers 模型是够会在每次前向计算的时候广播模型的缓冲区数据
- process_group 创建的进程组
- bucket_cap_mb 决定的是参数分块的大小
- find_unused_parameters True 找出模块中是否有一些参数没有在规约时被用到
- check_reduction 设置True， 保证在下一步的前向计算开始时，上一步的反向一定会完成


使用多进程时，为了防止输出竞争的产生，一般方法是判断当前的进程ID是否为0，如果是，则调用IO函数进行输出，否则不进行输出

In [ ]:
if not ard.multiprocessing_distributed or (args.multiprocessing_distributed and args.rank%ngpus_per_node==0):
    save()